데이콘 베이스라인 코드

In [2]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

light gbm이 뭐지?? LGBM regressor는 뭐지?

In [6]:
train = pd.read_csv('energy/train.csv',encoding='cp949')
test = pd.read_csv('energy/test.csv',encoding='cp949')
submission = pd.read_csv('energy/sample_submission.csv',encoding='cp949')

In [7]:
train
#train.shape 122400 X 10
#60개의 건물 X 85일 24시간 =122400

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [8]:
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


결측치를 보간한다??

In [9]:
#건물별로 '비전기 냉방설비운영' 과 '태양광 보유'를 판단하여 test set의 결측치를 보간해줍니다.
train[['num', '비전기냉방설비운영','태양광보유']]
ice = {}
hot = {}
count = 0
for i in range(0, len(train), len(train) // 60):
    count += 1
    ice[count] = train.loc[i, '비전기냉방설비운영']
    hot[count] = train.loc[i, '태양광보유']

In [10]:
for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유'] = hot[test['num'][i]]

In [11]:
#시간 변수와 요일 변수를 추가
def time(x):
    return int(x[-2:])

In [12]:
train['time'] = train['date_time'].apply(lambda x: time(x))
test['time'] = test['date_time'].apply(lambda x: time(x))

In [13]:
def weekday(x):
    return pd.to_datetime(x[:10]).weekday()

In [14]:
train['weekday'] = train['date_time'].apply(lambda x : weekday(x))
test['weekday'] = test['date_time'].apply(lambda x : weekday(x))

test 결측치를 보간해줌  
test 데이터의 변수는 예보 데이터이며, 예보 데이터는 train 데이터의 기간에 생성된 것이기에 활용 가능

In [15]:
test.interpolate(method='values') #진짜 보간이네??

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0,0,1
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0,1,1
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0,2,1
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0,3,1
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0,4,1
...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0,19,0
10076,60,2020-08-31 20,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0,20,0
10077,60,2020-08-31 21,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,21,0
10078,60,2020-08-31 22,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,22,0


# 모델링

In [16]:
train_x = train.drop('전력사용량(kWh)',axis=1)
train_y = train[['전력사용량(kWh)']]

In [17]:
train_x.drop('date_time',axis=1, inplace=True)
test.drop('date_time', axis=1, inplace=True)

In [18]:
cross = KFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in cross.split(train_x, train_y):
    folds.append((train_idx, valid_idx))

https://www.dacon.io/competitions/official/235713/codeshare/2476?page=1&dtype=recent

해당 방식으로 모델링 했다 하더라 나중에 함 참조해보자

파이썬 판다스(pandas) loc, iloc 차이  
판다스에서 데이터프레임을 특정 행, 열을 기준으로 나누고 싶을 때, loc, iloc을 주로 사용합니다.  
이 둘의 차이는 loc은 ‘변수명’을 기준으로 데이터프레임을 분리하고, iloc은 ‘인덱스번호’로 분류합니다.  

In [28]:
models = {}
count = 5

In [29]:

for fold in range(count):
    print(f'========================={fold+1}===================')
    train_idx, valid_idx = folds[fold]
    X_train = train_x.iloc[train_idx,:]
    y_train = train_y.iloc[train_idx,:]
    X_valid = train_x.iloc[valid_idx,:]
    y_valid = train_y.iloc[valid_idx,:]
    
    model = LGBMRegressor(n_estimators=100)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)],
             early_stopping_rounds=30, verbose=100)
    models[fold]=model
    
    print(f'============================================\n\n')

=========================1===================
Training until validation scores don't improve for 30 rounds
[100]	training's l2: 110589	valid_1's l2: 110225
Did not meet early stopping. Best iteration is:
[100]	training's l2: 110589	valid_1's l2: 110225


=========================2===================
Training until validation scores don't improve for 30 rounds
[100]	training's l2: 105803	valid_1's l2: 118195
Did not meet early stopping. Best iteration is:
[100]	training's l2: 105803	valid_1's l2: 118195


=========================3===================
Training until validation scores don't improve for 30 rounds
[100]	training's l2: 110710	valid_1's l2: 110163
Did not meet early stopping. Best iteration is:
[100]	training's l2: 110710	valid_1's l2: 110163


=========================4===================
Training until validation scores don't improve for 30 rounds
[100]	training's l2: 110637	valid_1's l2: 112547
Did not meet early stopping. Best iteration is:
[100]	training's l2: 110637	val

In [30]:
for i in range(count):
    submission['answer'] += models[i].predict(test) / count

In [31]:
submission

,num_date_time,answer
0,1 2020-08-25 00,25097.355492
1,1 2020-08-25 01,24594.897730
2,1 2020-08-25 02,24477.922011
3,1 2020-08-25 03,24958.716739
4,1 2020-08-25 04,24437.075227
...,...,...
10075,60 2020-08-31 19,8679.103160
10076,60 2020-08-31 20,7735.270933
10077,60 2020-08-31 21,8071.189901
10078,60 2020-08-31 22,6977.726170


In [32]:
submission.to_csv('energy/baseline_submission3.csv', index=False, encoding='cp949')